In [56]:
import random
import sys
import torch
import torch.nn as nn
from torch import cuda
from torch import optim
gpu_id = None

In [57]:
# model
class fittingRNN(nn.Module):
    def __init__(self, in_neurons = 2, hidden_neurons = 10, out_neurons = 1):
        super(fittingRNN, self).__init__()
        self.hidden_neurons = hidden_neurons
        self.W_x_h = nn.Linear(in_neurons, hidden_neurons)
        self.W_h_h = nn.Linear(hidden_neurons, hidden_neurons)
        self.W_h_y = nn.Linear(hidden_neurons, out_neurons)

        if gpu_id is not None:
            self.device = torch.device(gpu_id)
            self.to(self.device)
        else:
            self.device = torch.device('cpu')

        self.reset_state()

    def reset_state(self):
        self.h = torch.zeros(1, self.hidden_neurons).to(self.device)

    def forward(self, cur):
        self.h = torch.tanh(self.W_x_h(cur.to(self.device)) + self.W_h_h(self.h))
        y = self.W_h_y(self.h)
        return y

In [60]:
dataset = []
for _ in range(100):
    res = [[0, 0]]
    for i in range(100):
        nex = [random.randint(0, 1), res[-1][1] + random.uniform(-0.2, 0.2)]
        if res[-1][0] == 0:
            nex[1] += 1
        else:
            nex[1] -= 1
        res.append(nex)
    dataset.append(res)
print(dataset)

[[[0, 0], [1, 1.0671294571261427], [1, -0.10304186939925974], [0, -1.1112681104586697], [0, -0.03420620433358135], [1, 0.7943264908808805], [0, -0.27918770240511226], [1, 0.8966197384069625], [0, 0.017045876403594784], [1, 0.985936769614164], [0, -0.17909991515717194], [1, 0.9785476640685193], [1, -0.012069017129987647], [1, -1.009625453165345], [0, -2.174874405981507], [1, -1.3700146425170954], [0, -2.1722864921953793], [1, -1.189656048076801], [0, -2.167180610937291], [0, -1.1738471411688742], [1, -0.034205462924712604], [1, -1.1859220241438126], [1, -2.005853268041886], [0, -2.9281560581628088], [0, -2.059036166752585], [1, -1.0041944308728006], [0, -2.203417348944899], [1, -1.1894042401586584], [1, -2.1319916357793325], [0, -3.3188828961227625], [1, -2.2581219840119187], [0, -3.359525365444977], [1, -2.1773494521362498], [0, -3.138488446280891], [0, -1.9756894537264915], [0, -1.0995510726679902], [0, 0.01560630004767205], [1, 1.061338645169156], [0, 0.20190484655271512], [0, 1.0498

In [61]:
# train
model = fittingRNN()
loss_fn = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters())

epoch_num = 100
for epoch in range(epoch_num):
    print("{0} / {1} epoch start.".format(epoch + 1, epoch_num))
    sum_loss = 0.0
    for i, data in enumerate(dataset):
        model.reset_state()
        optimizer.zero_grad()
        accum_loss = None
        for cur, next in zip(data, data[1:] + [[0, 0]]):
            cur = torch.tensor([float(cur[0]), float(cur[1])])
            next = torch.tensor([float(next[1])]).unsqueeze(-1)
            out = model(cur)
            loss = loss_fn(out, next.to(out.device))
            accum_loss = loss if accum_loss is None else accum_loss + loss
        accum_loss.backward()
        optimizer.step()
        sum_loss += float(accum_loss.data.cpu())
        if (i + 1) % 100 == 0:
            print("{0} / {1} sentences finished.".format(i + 1, len(dataset)))
    print("mean loss = {0}.".format(sum_loss))

    model_file = "../trained_model/fitting_model_rnn_" + str(epoch + 1) + ".model"
    torch.save(model.state_dict(), model_file)

1 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 574870.7831726074.
2 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 477600.3893432617.
3 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 404899.19257354736.
4 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 347418.20836639404.
5 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 302940.8871078491.
6 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 267153.1365890503.
7 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 237656.93077087402.
8 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 212974.54055976868.
9 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 192049.2945690155.
10 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 174015.2602558136.
11 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 158294.98225736618.
12 / 100 epoch start.
100 / 100 sentences finished.
mean loss = 144565.89983701706.
13 / 10

In [63]:
# test
model = fittingRNN()
model.load_state_dict(torch.load("../trained_model/fitting_model_rnn_100.model"))
model.eval()

data = [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
# answer = [0.0, 1.0, 2.0, 3.0, 2.0, 3.0, 4.0, 3.0, 2.0, 1.0, 0.0, -1.0]
y = torch.tensor(0.0)
x = torch.tensor([data[0], y])
for i in range(1, len(data)):
    y = model(x)
    print(y)
    x = torch.tensor([data[i], y])

tensor([[1.0150]], grad_fn=<AddmmBackward0>)
tensor([[1.9598]], grad_fn=<AddmmBackward0>)
tensor([[3.0247]], grad_fn=<AddmmBackward0>)
tensor([[2.0262]], grad_fn=<AddmmBackward0>)
tensor([[2.9525]], grad_fn=<AddmmBackward0>)
tensor([[3.8948]], grad_fn=<AddmmBackward0>)
tensor([[2.8498]], grad_fn=<AddmmBackward0>)
tensor([[1.8096]], grad_fn=<AddmmBackward0>)
tensor([[0.8202]], grad_fn=<AddmmBackward0>)
tensor([[-0.1714]], grad_fn=<AddmmBackward0>)
tensor([[-1.1484]], grad_fn=<AddmmBackward0>)
